In [1]:
import os
import shutil

# Define the source and target directories
source_dir = '/kaggle/input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign'  # Replace with the correct path
target_dir = '/kaggle/working/data/benign_flattened'  # Create a new folder for all images


# Ensure the target directory exists
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Walk through all subdirectories of the benign folder
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):  # Add other image formats if needed
            source_path = os.path.join(root, file)
            target_path = os.path.join(target_dir, file)
            
            # Handle duplicate filenames by renaming
            base, ext = os.path.splitext(file)
            counter = 1
            while os.path.exists(target_path):
                target_path = os.path.join(target_dir, f"{base}_{counter}{ext}")
                counter += 1

            # Copy the file to the target directory and then remove the original file
            shutil.copy2(source_path, target_path)
            

print("All images have been copied and moved to:", target_dir)

All images have been copied and moved to: /kaggle/working/data/benign_flattened


In [2]:
source_dir = '/kaggle/input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant'  # Replace with the correct path
target_dir = '/kaggle/working/data/malignant_flattened'  # Create a new folder for all images


# Ensure the target directory exists
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Walk through all subdirectories of the benign folder
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):  # Add other image formats if needed
            source_path = os.path.join(root, file)
            target_path = os.path.join(target_dir, file)
            
            # Handle duplicate filenames by renaming
            base, ext = os.path.splitext(file)
            counter = 1
            while os.path.exists(target_path):
                target_path = os.path.join(target_dir, f"{base}_{counter}{ext}")
                counter += 1

            # Copy the file to the target directory and then remove the original file
            shutil.copy2(source_path, target_path)
            

print("All images have been copied and moved to:", target_dir)


All images have been copied and moved to: /kaggle/working/data/malignant_flattened


In [3]:
import shutil
shutil.make_archive('/kaggle/working/data', 'zip')


'/kaggle/working/data.zip'

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size you want to resize to (adjust based on model input size)
IMG_SIZE = (224, 224)

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalization
    rotation_range=30,            # Random rotation
    width_shift_range=0.2,        # Random width shift
    height_shift_range=0.2,       # Random height shift
    shear_range=0.2,              # Random shear
    zoom_range=0.2,               # Random zoom
    horizontal_flip=True,         # Random horizontal flip
    fill_mode='nearest',          # Fill in missing pixels
    validation_split=0.2          # Set 20% of data as validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    'data/',                      # Path to data directory
    target_size=IMG_SIZE,         # Resize images
    batch_size=32,                # Batch size
    class_mode='binary',          # Binary classification
    subset='training',            # Specify that this is for training data
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    'data/',                      # Path to data directory
    target_size=IMG_SIZE,         # Resize images
    batch_size=32,                # Batch size
    class_mode='binary',          # Binary classification
    subset='validation',          # Specify that this is for validation data
    shuffle=False
)


Found 6328 images belonging to 2 classes.
Found 1581 images belonging to 2 classes.


In [6]:
def build_cnn_model(input_shape=(224, 224, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and summarize the CNN model
cnn_model = build_cnn_model()
cnn_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
cnn_model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate on validation data
y_true = validation_generator.classes
y_pred = cnn_model.predict(validation_generator)
y_pred = (y_pred > 0.5)  

# Metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc_roc = roc_auc_score(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


198/198 ━━━━━━━━━━━━━━━━━━━━ 171s 807ms/step - accuracy: 0.7358 - loss: 0.7346 - val_accuracy: 0.7438 - val_loss: 0.4912
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 770ms/step - accuracy: 0.7717 - loss: 0.4884 - val_accuracy: 0.8318 - val_loss: 0.4263
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 769ms/step - accuracy: 0.8259 - loss: 0.4450 - val_accuracy: 0.8368 - val_loss: 0.3754
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 770ms/step - accuracy: 0.8307 - loss: 0.4425 - val_accuracy: 0.8204 - val_loss: 0.4244
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 773ms/step - accuracy: 0.8147 - loss: 0.4601 - val_accuracy: 0.7849 - val_loss: 0.4783
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 771ms/step - accuracy: 0.8182 - loss: 0.4492 - val_accuracy: 0.7774 - val_loss: 0.4651
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 769ms/step - accuracy: 0.8328 - loss: 0.4243 - val_accuracy: 0.7824 - val_loss: 0.4937
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 770ms/step - accuracy: 0.8095 - loss: 0.46

In [8]:
def build_transfer_learning_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and summarize the transfer learning model
transfer_model = build_transfer_learning_model()
transfer_model.summary()

# Train the model
transfer_model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate on validation data
y_pred = transfer_model.predict(validation_generator)
y_pred = (y_pred > 0.5)  

# Metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc_roc = roc_auc_score(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 822ms/step - accuracy: 0.6680 - loss: 0.6494 - val_accuracy: 0.6863 - val_loss: 0.6222
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 158s 774ms/step - accuracy: 0.6973 - loss: 0.6153 - val_accuracy: 0.6863 - val_loss: 0.6545
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 158s 774ms/step - accuracy: 0.6907 - loss: 0.6213 - val_accuracy: 0.6863 - val_loss: 0.6938
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 772ms/step - accuracy: 0.6926 - loss: 0.6299 - val_accuracy: 0.6781 - val_loss: 0.6250
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 158s 776ms/step - accuracy: 0.7059 - loss: 0.6019 - val_accuracy: 0.6679 - val_loss: 0.6284
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 158s 774ms/step - accuracy: 0.7044 - loss: 0.5976 - val_accuracy: 0.6755 - val_loss: 0.6253
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 773ms/step - accuracy: 0.7132 - loss: 0.5894 - val_accuracy: 0.6755 - val_loss: 0.6237
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 157s 770ms/step - accuracy: 0.7025 -

In [9]:
import kerastuner as kt

def build_model_with_tuning(hp):
    model = models.Sequential([
        layers.Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(hp.Int('units', min_value=64, max_value=256, step=64), activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'sgd']), loss='binary_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model_with_tuning, objective='val_accuracy', max_epochs=10, factor=3)
tuner.search(train_generator, validation_data=validation_generator)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Trial 6 Complete [00h 00m 20s]

Best val_accuracy So Far: 0.824161946773529
Total elapsed time: 00h 17m 20s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
64                |64                |filters
256               |192               |units
sgd               |adam              |optimizer
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _t

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/pyth

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define base directory
base_dir = "/kaggle/input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast"

# Load all images and labels
def load_data(base_dir):
    images = []
    labels = []
    
    for label, class_name in enumerate(["benign", "malignant"]):
        class_dir = os.path.join(base_dir, class_name, "SOB")
        for subtype in os.listdir(class_dir):
            subtype_dir = os.path.join(class_dir, subtype)
            for patient in os.listdir(subtype_dir):
                patient_dir = os.path.join(subtype_dir, patient)
                for mag in ["40X", "100X", "200X", "400X"]:
                    mag_dir = os.path.join(patient_dir, mag)
                    if os.path.exists(mag_dir):
                        for img_file in os.listdir(mag_dir):
                            if img_file.endswith(".png"):
                                img_path = os.path.join(mag_dir, img_file)
                                img = cv2.imread(img_path)
                                img = cv2.resize(img, (224, 224))  # Resize for model input
                                images.append(img)
                                labels.append(label)  # 0 for benign, 1 for malignant
    
    return np.array(images), np.array(labels)

# Load dataset
X, y = load_data(base_dir)

# Normalize images
X = X / 255.0

# Split dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True, zoom_range=0.2)
datagen.fit(X_train)

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

datagen = ImageDataGenerator(rescale=1./255)  # Assuming you're using data augmentation

def plot_history(history, model_name):
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

def train_model(model, name, X_train, y_train, X_val, y_val, X_test, y_test):
    model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                        validation_data=(X_val, y_val),
                        epochs=10, verbose=1)
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"{name} Test Accuracy: {test_acc}")
    plot_history(history, name)
    return model, history, test_acc

# ResNet50
base_resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)
resnet_model = Model(inputs=base_resnet.input, outputs=predictions)
for layer in base_resnet.layers:
    layer.trainable = False  # Freeze base layers initially

# VGG16
base_vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)
vgg_model = Model(inputs=base_vgg.input, outputs=predictions)
for layer in base_vgg.layers:
    layer.trainable = False

# Custom CNN
custom_model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Train all models and plot results
models = {"ResNet50": resnet_model, "VGG16": vgg_model, "CustomCNN": custom_model}
results = {}
for name, model in models.items():
    trained_model, history, test_acc = train_model(model, name, X_train, y_train, X_val, y_val, X_test, y_test)
    results[name] = {"model": trained_model, "history": history, "test_acc": test_acc}


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


173/173 ━━━━━━━━━━━━━━━━━━━━ 37s 147ms/step - accuracy: 0.6714 - loss: 0.6402 - val_accuracy: 0.6863 - val_loss: 0.6204
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.6867 - loss: 0.6315 - val_accuracy: 0.6863 - val_loss: 0.6214
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 18s 103ms/step - accuracy: 0.6806 - loss: 0.6357 - val_accuracy: 0.6863 - val_loss: 0.6279
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.6799 - loss: 0.6389 - val_accuracy: 0.6863 - val_loss: 0.6246
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 20s 115ms/step - accuracy: 0.6948 - loss: 0.6259 - val_accuracy: 0.6863 - val_loss: 0.6211
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 20s 116ms/step - accuracy: 0.6763 - loss: 0.6353 - val_accuracy: 0.6863 - val_loss: 0.6285
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.6861 - loss: 0.6279 - val_accuracy: 0.6863 - val_loss: 0.6211
Epoch 8/10
